In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage

In [2]:
'''
Split the provided training data to create a new training 
data set and a new validation data set. These will be used
or hyper-parameter tuning.
'''

# For reproducibility
seed = 27

raw_data = pd.read_csv("raw_data/train.csv")

train, validate = train_test_split(raw_data, test_size=0.2, random_state = seed, stratify = raw_data['label'])

train.to_csv("modified_data/train.csv")
validate.to_csv("modified_data/validate.csv")

In [3]:
'''
Expand the above training data to use simple transformations 
of one pixel left, right, up, and down. Hopefully this will
also include small rotations of 1 - 3 degrees in each direction
in the not too distant future.
'''

# Reshape a 1D vector of an mnist digit to 2D
def reshape_digit_square(mnist_pixels):
    reshaped_digit = mnist_pixels.reshape(28, 28)
    return reshaped_digit


# Reshape a 2D array of an mnist digit to 1D
def reshape_digit_flat(mnist_pixels):
    reshaped_digit = mnist_pixels.reshape(28*28)
    return reshaped_digit


# Shift the digit data by an arbitrary amount up or down and left or right
def shift_digit(mnist_pixels, x_shift, y_shift):
    mnist_pixels = reshape_digit_square(mnist_pixels)
    
    padding = ((max([-y_shift, 0]), max([y_shift, 0])),(max([x_shift,0]), max([-x_shift, 0])))
    y1 = max([y_shift, 0])
    y2 = max([mnist_pixels.shape[0], mnist_pixels.shape[0] + y_shift])
    x1 = max([-x_shift, 0])
    x2 = max([mnist_pixels.shape[1], mnist_pixels.shape[1] - x_shift])
    
    new_digit = np.pad(mnist_pixels,padding, mode='constant')[y1:y2, x1:x2]
    new_digit = reshape_digit_flat(new_digit)
    
    return new_digit

# Move each digit in the data set the same way
def move_mnist_digits(mnist_data, x_shift, y_shift):
    pixel_data = mnist_data.iloc[:, 1:].as_matrix()
    new_pixels = np.apply_along_axis(shift_digit, 1, pixel_data, x_shift, y_shift)
    new_data = mnist_data.iloc[:, 0]
    new_data = new_data.values.reshape(new_data.shape[0], 1)
    new_data = np.concatenate((new_data, new_pixels), axis = 1)
    new_data = pd.DataFrame(new_data, columns = list(mnist_data))
    return new_data


# Rotate the data by an arbitrary amount
def rotate_digit(mnist_pixels, angle):
    mnist_pixels = reshape_digit_square(mnist_pixels)
    new_digit = ndimage.interpolation.rotate(mnist_pixels, angle, reshape = False)
    new_digit = reshape_digit_flat(new_digit)
    return new_digit

# Rotate each digit in the data set the same way
def rotate_mnist_digits(mnist_data, angle):
    pixel_data = mnist_data.iloc[:, 1:].as_matrix()
    new_pixels = np.apply_along_axis(rotate_digit, 1, pixel_data, angle)
    new_data = mnist_data.iloc[:, 0]
    new_data = new_data.values.reshape(new_data.shape[0], 1)
    new_data = np.concatenate((new_data, new_pixels), axis = 1)
    new_data = pd.DataFrame(new_data, columns = list(mnist_data))
    return new_data

# Skew the data by an arbitrary amount
def skew_mnist(mnist_data, angle):
    return

In [4]:
# Shift the training data one pixel in each direction
train_up_one = move_mnist_digits(train, 0, 1)
train_down_one = move_mnist_digits(train, 0, -1)
train_left_one = move_mnist_digits(train, -1, 0)
train_right_one = move_mnist_digits(train, 1, 0)
train_rotate_cw = rotate_mnist_digits(train, -6)
train_rotate_ccw = rotate_mnist_digits(train, 6)
frames = [train, train_up_one, train_down_one, train_left_one, \
          train_right_one, train_rotate_cw, train_rotate_ccw]

In [5]:
# Write the expanded training data to file
train_expanded = pd.concat(frames)
train_expanded.to_csv("modified_data/train_expanded.csv")

In [6]:
'''before = train.values[0, 1:].reshape(28,28)
plt.imshow(before, cmap='gray')
plt.show()'''

"before = train.values[0, 1:].reshape(28,28)\nplt.imshow(before, cmap='gray')\nplt.show()"

In [7]:
'''results = rotate_mnist_digits(train, -12)
after = results.values[0, 1:].reshape(28,28)
plt.imshow(after, cmap='gray')
plt.show()'''

"results = rotate_mnist_digits(train, -12)\nafter = results.values[0, 1:].reshape(28,28)\nplt.imshow(after, cmap='gray')\nplt.show()"